In [158]:
import pandas as pd
from transformers import AutoModel, AutoTokenizer
from torch import nn
import torch.nn.functional as F
from sklearn.metrics import accuracy_score, f1_score
from torch.utils.data import DataLoader, Dataset

In [159]:
data = pd.read_excel("/content/drive/MyDrive/젯봇/data.xlsx", engine="openpyxl")#.iloc[:1000]

In [160]:
#data.shape

In [161]:
all_indices = data.index

In [162]:
#len(all_indices)

In [163]:
train_size = int(0.8 * len(all_indices))
val_size = len(all_indices) - train_size

train_idx, val_idx = all_indices[:int(0.8 * len(all_indices))], all_indices[int(0.8*len(all_indices)):]

In [164]:
# 인덱스는 0부터 시작하니까 클래스는 +1을 해줘야한다.
num_classes = max(data["label_idx"].tolist())+1

In [165]:
#train_idx, val_idx

In [166]:
import torch
from torch.utils.data import DataLoader, Dataset, random_split
from transformers import BertForSequenceClassification, BertTokenizer, AdamW
from transformers import get_constant_schedule_with_warmup
from tqdm.auto import tqdm

tokenizer = AutoTokenizer.from_pretrained("klue/roberta-small")
model = AutoModel.from_pretrained("klue/roberta-small")

Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-small and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [167]:
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [168]:
# 데이터셋 클래스 정의
class TextClassificationDatasetBatch(Dataset):
    def __init__(self, df, tokenizer, batch_size=32):
        self.tokenizer = tokenizer
        self.sentences = df["sentence"].tolist()
        self.labels = df["label_idx"].tolist()

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        # to_return = {
        #     "sentence_encoded":self.encodings[idx],
        #     "label":self.labels[idx]
        # }
        return (self.sentences[idx], self.labels[idx])

In [169]:
train_data = data.iloc[train_idx]
val_data = data.iloc[val_idx]

In [170]:
train_dataset = TextClassificationDatasetBatch(train_data, tokenizer)
val_dataset = TextClassificationDatasetBatch(val_data, tokenizer)

In [171]:
#train_dataset[0]

In [172]:
def collate_bert_tokenizer(samples):
  sentences, labels = [s[0]for s in samples], [s[1] for s in samples]
  global tokenizer
  return tokenizer(sentences, return_tensors="pt", padding=True, truncation=True, max_length=512),labels

In [173]:
batch_size = 32
train_dataloader = DataLoader(train_dataset, batch_size = batch_size, shuffle=True, collate_fn = collate_bert_tokenizer)
val_dataloader = DataLoader(val_dataset, batch_size = batch_size, shuffle=False, collate_fn = collate_bert_tokenizer)

In [174]:
#sentences = data["sentence"].tolist()
#tokenized = tokenizer(sentences, return_tensors="pt", padding=True, truncation=True, max_length=512)
#output = model(**tokenized)``

In [175]:
#print(tokenized["input_ids"].shape) # 43은 각 문장의 토큰 수 `

In [176]:
#model(**tokenized)["pooler_output"].shape # 768은 고차원벡터 / 768차원 벡터라는 뜻

In [177]:
#tokenized.keys()

In [178]:
#model(input_ids=tokenized["input_ids"], token_type_ids=tokenized["token_type_ids"], attention_mask=tokenized["attention_mask"])

In [184]:
class CustomModel(nn.Module):
  def __init__(self, bert_model, num_classes, hidden_size):
    super(CustomModel, self).__init__()
    self.model = bert_model
    self.num_classes = num_classes

    self.fc = nn.Linear(hidden_size, num_classes)

  def forward(self, _input):
    output = self.model(**_input)["pooler_output"]
    output = self.fc(output)
    return output

custom_model = CustomModel(model, num_classes, 768)
optimizer = AdamW(custom_model.parameters(),lr=0.003)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [180]:
import torch.optim as optim

In [181]:
#num_train_samples = len(train_dataloader.dataset)
#print(f"Number of samples in the training dataset: {num_train_samples}")

In [182]:
#num_val_samples = len(val_dataloader.dataset)
#print(f"Number of samples in the validation dataset: {num_val_samples}")

In [185]:
loss_f = nn.CrossEntropyLoss()

epochs = 4

for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")
    print('-' * 10)

    # 모델을 학습 모드로 설정
    custom_model.train()
    total_loss = 0.0

    for i, (_input, _label) in enumerate(tqdm(train_dataloader, desc=f"Epoch {epoch + 1}")):
        _label = torch.tensor(F.one_hot(torch.tensor(_label, dtype=torch.long), num_classes=num_classes), dtype=torch.float).squeeze()

        outputs = custom_model(_input)
        loss = loss_f(outputs, _label.argmax(dim=1).long())
        loss.backward()  # 역전파 수행
        optimizer.step()
        optimizer.zero_grad()

        total_loss += loss.item()

    average_loss = total_loss / len(train_dataloader)
    print(f"Epoch {epoch + 1} - Average Loss : {average_loss:.4f}")

    # 모델 평가
    custom_model.eval()
    all_predictions = []
    all_labels = []

    with torch.no_grad():
        for _input, _label in tqdm(val_dataloader, desc=f"Evaluation Epoch {epoch + 1}"):
            _label = torch.tensor(F.one_hot(torch.tensor(_label, dtype=torch.long), num_classes=num_classes), dtype=torch.float).squeeze()

            output = custom_model(_input)
            predictions = output.argmax(dim=1)

            all_predictions.extend(predictions.cpu().numpy())
            all_labels.extend(_label.argmax(dim=1).cpu().numpy())

        f1 = f1_score(all_labels, all_predictions, average='micro')
        accuracy = accuracy_score(all_labels, all_predictions)

        print(f"Epoch {epoch + 1} - F1 Score: {f1:.4f}, Accuracy: {accuracy:.4f}")

Epoch 1/4
----------


Epoch 1:   0%|          | 0/394 [00:00<?, ?it/s]

<ipython-input-185-506a482cc2d3>:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  _label = torch.tensor(F.one_hot(torch.tensor(_label, dtype=torch.long), num_classes=num_classes), dtype=torch.float).squeeze()


Epoch 1 - Average Loss : 9.1560


Evaluation Epoch 1:   0%|          | 0/99 [00:00<?, ?it/s]

...

<ipython-input-185-506a482cc2d3>:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  _label = torch.tensor(F.one_hot(torch.tensor(_label, dtype=torch.long), num_classes=num_classes), dtype=torch.float).squeeze()


Epoch 4 - Average Loss : 8.9532


Evaluation Epoch 4:   0%|          | 0/99 [00:00<?, ?it/s]

<ipython-input-185-506a482cc2d3>:34: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  _label = torch.tensor(F.one_hot(torch.tensor(_label, dtype=torch.long), num_classes=num_classes), dtype=torch.float).squeeze()


Epoch 4 - F1 Score: 0.2463, Accuracy: 0.2463
